# Local Data Tests

### Import Modules

In [1]:
import numpy as np
import xarray as xr
import timeit
import matplotlib.pyplot as plt
from dask.distributed import Client

## Setup for Global Tests

### Open MUR Global Dataset

In [ ]:
start_time = timeit.default_timer()

mur_global = xr.open_mfdataset(
    '../data/podaac_web_global/*.nc4', 
    engine='netcdf4',
    chunks={"time": 30, "lat": 100, "lon": 100}
)

elapsed = timeit.default_timer() - start_time
print(elapsed)

In [ ]:
mur_global

### Add in NAN Values for Land to MUR Data
<br/>
We use the mask dimension to replace temperature values from land observations with NaN so that they are not factored in to our calculations. The mask variable has a value for each coordinate pair representing which surface the temperature was collected from (land, open-sea, ice, etc.).

In [ ]:
mur_global_sst = mur_global['analysed_sst'].where(mur.mask == 1)

### Convert Temperatures to Celsius
<br/>
The dataset is stored with temperatures measured in Kelvin. This converts it to Celsius for ease of understanding and analysis.

In [ ]:
mur_global_sst = mur_global_sst - 273.15

In [ ]:
mur_global_sst

### Find Daily Average Temperature for Time Series

In [ ]:
mur_global_sst_mean_ts = mur_global_sst.mean(['lat', 'lon'])

### Find Average Temperature for Each Coordinate Pair for Spatial Plot

In [ ]:
mur_global_sst_mean_sp = mur_global_sst.mean(['time'])

## Global Tests

### Global SST Averaged Time Series

In [ ]:
start_time = timeit.default_timer()

mur_global_sst_mean_ts.plot()

elapsed = timeit.default_timer() - start_time
print(elapsed)

### Global SST Averaged Spatial Plot

In [ ]:
start_time = timeit.default_timer()

mur_global_sst_mean_sp.plot()

elapsed = timeit.default_timer() - start_time
print(elapsed)

## Setup for Regional Tests

In [ ]:
start_time = timeit.default_timer()

mur_hawaii = xr.open_mfdataset(
    '../data/podaac_web_hawaii/*.nc4', 
    engine='netcdf4',
    chunks={"time": 30, "lat": 100, "lon": 100}
)

elapsed = timeit.default_timer() - start_time
print(elapsed)

In [ ]:
mur_hawaii

### Add in NAN Values for Land to MUR Data
<br/>
We use the mask dimension to replace temperature values from land observations with NaN so that they are not factored in to our calculations. The mask variable has a value for each coordinate pair representing which surface the temperature was collected from (land, open-sea, ice, etc.).

In [22]:
mur_hawaii_sst = mur_hawaii['analysed_sst'].where(mur_hawaii.mask == 1)

### Convert Temperatures to Celsius
<br/>
The dataset is stored with temperatures measured in Kelvin. This converts it to Celsius for ease of understanding and analysis.

In [23]:
mur_hawaii_sst = mur_hawaii_sst - 273.15

### Open MUR Climatology for Hawaii

In [ ]:
mur_clim = xr.open_dataarray(
    "../data/MURClimatology.nc", 
    chunks={"time": 30, "lat": 100, "lon": 100}
)

In [ ]:
mur_clim

### Drop the Leap Day

In [ ]:
mur_clim = mur_clim.where(mur_clim["time"] != np.datetime64('2004-02-29T09:00:00', 'ns'), drop=True)

### Create Subset

In [39]:
mur_clim_jan = mur_clim[0:20]

In [40]:
mur_clim_subset = mur_clim[212:]

In [41]:
mur_clim_subset = xr.concat([mur_clim_subset, mur_clim_jan], dim="time")

In [43]:
mur_clim_subset = mur_clim_subset.assign_coords({"time": mur_hawaii_sst["time"]})

In [44]:
mur_clim_subset

,Array,Chunk
Bytes,485.58 MB,1.20 MB
Shape,"(173, 701, 1001)","(30, 100, 100)"
Count,4692 Tasks,616 Chunks
Type,float32,numpy.ndarray


### Create SST Anomaly Dataset

In [ ]:
sst_anomaly = mur_hawaii - mur_clim_subset

### Find Daily Average Temperature for Time Series

In [ ]:
sst_anomaly_mean_ts = sst_anomaly.mean(['lat', 'lon'])

### Find Average Temperature for Each Coordinate Pair for Spatial Plot

In [ ]:
sst_anomaly_mean_sp = sst_anomaly.mean(['time'])

## Regional Tests

### Regional SST Anomaly Averaged Time Series, August 2019 - January 2020

In [ ]:
start_time = timeit.default_timer()

sst_anomaly_mean_ts

elapsed = timeit.default_timer() - start_time
print(elapsed)

### Regional SST Anomaly Averaged Spatial Plot, August 2019 - January 2020

In [ ]:
start_time = timeit.default_timer()

sst_anomaly_mean_sp

elapsed = timeit.default_timer() - start_time
print(elapsed)